In [2]:
from pyspark.sql import SparkSession, Row
import numpy as np
import pandas as pd
import feather
from pyspark.ml.stat import Correlation
#from pyspark.sql.functions import * 
from pyspark.sql.functions import avg
import pyspark.sql.functions as fcn 
from pyspark.sql.functions import pandas_udf, PandasUDFType
from pyspark.sql.types import IntegerType, StringType, TimestampType
#------------------------------------------
from sklearn.metrics import accuracy_score
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.ml.feature import StringIndexer, OneHotEncoder
from pyspark.ml.regression import RandomForestRegressor
#from pyspark.ml.evaluation import BinaryClassificationEvaluator
#from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml import Pipeline

In [3]:
spark = SparkSession.builder.appName('Bulldozer').getOrCreate()

# Load the data 
### Convention : use 'r' for 'raw'

In [49]:
# CV Set
# Description of the abbreviations 
# rdt: raw data  # prs: parasitics (may include columns that will be deleted) # rlbl: raw label

rdt_t_prs_rlbl = spark.read.csv('train.csv', inferSchema=True, header=True)   
#===============================================================

# Test Set
rdt_T_t_prs = spark.read.csv('Valid.csv', inferSchema=True, header=True) 

# No. of Rows and Columns 

In [1]:
print('Num of rows in the training set: ' , rdt_t_prs_rlbl.count())
print('Num of columns in the training set: ' , len(rdt_t_prs_rlbl.columns))
print('-----------------------------------------')
print('Num of rows in the validation set: ' , rdt_T_t_prs.count())
print('Num of columns in the validation set: ' , len(rdt_T_prs.columns))

NameError: name 'rdt_t_prs_rlbl' is not defined

# Print Schema

In [51]:
rdt_t_prs_rlbl.printSchema()

# Change column names if necessary 
'''
for name in df.schema.names:
  df = df.withColumnRenamed(name, name.replace(' ', ''))'''

root
 |-- SalesID: integer (nullable = true)
 |-- SalePrice: integer (nullable = true)
 |-- MachineID: integer (nullable = true)
 |-- ModelID: integer (nullable = true)
 |-- datasource: integer (nullable = true)
 |-- auctioneerID: integer (nullable = true)
 |-- YearMade: integer (nullable = true)
 |-- MachineHoursCurrentMeter: integer (nullable = true)
 |-- UsageBand: string (nullable = true)
 |-- saledate: string (nullable = true)
 |-- fiModelDesc: string (nullable = true)
 |-- fiBaseModel: string (nullable = true)
 |-- fiSecondaryDesc: string (nullable = true)
 |-- fiModelSeries: string (nullable = true)
 |-- fiModelDescriptor: string (nullable = true)
 |-- ProductSize: string (nullable = true)
 |-- fiProductClassDesc: string (nullable = true)
 |-- state: string (nullable = true)
 |-- ProductGroup: string (nullable = true)
 |-- ProductGroupDesc: string (nullable = true)
 |-- Drive_System: string (nullable = true)
 |-- Enclosure: string (nullable = true)
 |-- Forks: string (nullable =

"\nfor name in df.schema.names:\n  df = df.withColumnRenamed(name, name.replace(' ', ''))"

# Perform operations on the label column if necessary

In [53]:
rdt_t_prs = rdt_t_prs_rlbl.withColumn('logSalePrice',log(rdt_t_prs_rlbl.SalePrice)).drop('SalePrice')
rdt_t_prs.printSchema()

root
 |-- SalesID: integer (nullable = true)
 |-- MachineID: integer (nullable = true)
 |-- ModelID: integer (nullable = true)
 |-- datasource: integer (nullable = true)
 |-- auctioneerID: integer (nullable = true)
 |-- YearMade: integer (nullable = true)
 |-- MachineHoursCurrentMeter: integer (nullable = true)
 |-- UsageBand: string (nullable = true)
 |-- saledate: string (nullable = true)
 |-- fiModelDesc: string (nullable = true)
 |-- fiBaseModel: string (nullable = true)
 |-- fiSecondaryDesc: string (nullable = true)
 |-- fiModelSeries: string (nullable = true)
 |-- fiModelDescriptor: string (nullable = true)
 |-- ProductSize: string (nullable = true)
 |-- fiProductClassDesc: string (nullable = true)
 |-- state: string (nullable = true)
 |-- ProductGroup: string (nullable = true)
 |-- ProductGroupDesc: string (nullable = true)
 |-- Drive_System: string (nullable = true)
 |-- Enclosure: string (nullable = true)
 |-- Forks: string (nullable = true)
 |-- Pad_Type: string (nullable = t

# Identify the Label and Index columns

In [5]:
LBL = 'logSalePrice'
IDX = 'SalesID'

# ==========================

# Handling datetime 

# ==========================

### Define the Pandas function to convert the string series to timestamp

In [55]:
def pd_StringToTime(s):    
    '''
    The current format to be converted is: 'YYYY/M/D 0:00'
    '''
    return pd.to_datetime(s)

### Define the pandas_udf function

In [56]:
# Syntax: pyspark.sql.functions.pandas_udf(f=None, returnType=None, functionType=None)[source]
Time_udf = pandas_udf(pd_StringToTime, TimestampType())   

### Apply the pandas_udf function to timeseries (string type) of the orignial dataframe 
### Add time-features 

In [61]:
# CV set 

rdt_prs = rdt_t_prs.withColumn('Sale_Date',    Time_udf(rdt_t_prs.saledate)).drop('saledate')\
                      .withColumn('yDay', dayofyear('Sale_Date'))\
                      .withColumn('mDay', dayofmonth('Sale_Date'))\
                      .withColumn('wDay', dayofweek('Sale_Date')).drop('Sale_Date')

# ================================================================================================


# Test set 
rdt_T_prs = rdt_T_t_prs.withColumn('Sale_Date', Time_udf(rdt_T_t_prs.saledate)).drop('saledate')\
                      .withColumn('yDay', dayofyear('Sale_Date'))\
                      .withColumn('mDay', dayofmonth('Sale_Date'))\
                      .withColumn('wDay', dayofweek('Sale_Date')).drop('Sale_Date')




# Have a look at the first few rows

In [60]:
pd.set_option('display.max_columns', 100)

pd.DataFrame(rdt_prs.head(8))



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54
0,1139246,999089,3157,121,3,2004,68,Low,521D,521,D,None,None,None,Wheel Loader - 110.0 to 120.0 Horsepower,Alabama,WL,Wheel Loader,None,EROPS w AC,None or Unspecified,None,None or Unspecified,None,None,None,None,None,None,None,2 Valve,None,None,None,None,None or Unspecified,None or Unspecified,None,None,None,None,None,None,None,None,None,None,None,None,Standard,Conventional,11.097410,320,16,5
1,1139248,117657,77,121,3,1996,4640,Low,950FII,950,F,II,None,Medium,Wheel Loader - 150.0 to 175.0 Horsepower,North Carolina,WL,Wheel Loader,None,EROPS w AC,None or Unspecified,None,None or Unspecified,None,None,None,None,None,None,None,2 Valve,None,None,None,None,23.5,None or Unspecified,None,None,None,None,None,None,None,None,None,None,None,None,Standard,Conventional,10.950807,86,26,6
2,1139249,434808,7009,121,3,2001,2838,High,226,226,None,None,None,None,Skid Steer Loader - 1351.0 to 1601.0 Lb Operat...,New York,SSL,Skid Steer Loaders,None,OROPS,None or Unspecified,None,None,None,None,None,None,None,None,None,Auxiliary,None,None,None,None,None,None or Unspecified,None or Unspecified,None or Unspecified,Standard,None,None,None,None,None,None,None,None,None,None,None,9.210340,57,26,5
3,1139251,1026470,332,121,3,2001,3486,High,PC120-6E,PC120,None,-6E,None,Small,"Hydraulic Excavator, Track - 12.0 to 14.0 Metr...",Texas,TEX,Track Excavators,None,EROPS w AC,None,None,None,None,None,None,None,None,None,None,2 Valve,None,None,None,None,None,None or Unspecified,None,None,None,None,None,None,None,None,None,None,None,None,None,None,10.558414,139,19,5
4,1139253,1057373,17311,121,3,2007,722,Medium,S175,S175,None,None,None,None,Skid Steer Loader - 1601.0 to 1751.0 Lb Operat...,New York,SSL,Skid Steer Loaders,None,EROPS,None or Unspecified,None,None,None,None,None,None,None,None,None,Auxiliary,None,None,None,None,None,None or Unspecified,None or Unspecified,None or Unspecified,Standard,None,None,None,None,None,None,None,None,None,None,None,9.305651,204,23,5
5,1139255,1001274,4605,121,3,2004,508,Low,310G,310,G,None,None,None,Backhoe Loader - 14.0 to 15.0 Ft Standard Digg...,Arizona,BL,Backhoe Loaders,Four Wheel Drive,OROPS,None or Unspecified,None or Unspecified,No,Extended,Powershuttle,None or Unspecified,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,10.184900,353,18,5
6,1139256,772701,1937,121,3,1993,11540,High,790ELC,790,E,None,LC,Large / Medium,"Hydraulic Excavator, Track - 21.0 to 24.0 Metr...",Florida,TEX,Track Excavators,None,EROPS,None,None,None,None,None,None,None,None,None,None,Standard,None,None,None,None,None,None or Unspecified,None,None,None,Steel,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,None,None,None,None,None,9.952278,239,26,5
7,1139261,902002,3539,121,3,2001,4883,High,416D,416,D,None,None,None,Backhoe Loader - 14.0 to 15.0 Ft Standard Digg...,Illinois,BL,Backhoe Loaders,Four Wheel Drive,OROPS,None or Unspecified,Reversible,No,Standard,Standard,Yes,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,10.203592,321,17,5


# Check the percentage of Null values in each column

## CV Set Null counts 

In [9]:
NullCheck = rdt_prs.select([bround(((sum(when(isnull(c), 1).otherwise(0)))/rdt_prs.count()*100),2).alias(c)\
                   for c in rdt_prs.columns])

In [10]:
counter = 0
thresh = 50
for c in NullCheck.columns:
    if NullCheck.select(c).collect()[0][0]>thresh:
        print(c,NullCheck.select(c).collect()[0][0])  
        counter +=1
print('No. of columns with more than {}% Null values; '.format(thresh), counter) 

MachineHoursCurrentMeter 64.41
UsageBand 82.64
fiModelSeries 85.81
fiModelDescriptor 82.07
ProductSize 52.55
Drive_System 73.98
Forks 52.12
Pad_Type 80.27
Ride_Control 62.95
Stick 80.27
Transmission 54.32
Turbocharged 80.27
Blade_Extension 93.71
Blade_Width 93.71
Enclosure_Type 93.71
Engine_Horsepower 93.71
Pushblock 93.71
Ripper 74.04
Scarifier 93.71
Tip_Control 93.71
Tire_Size 76.39
Coupler_System 89.17
Grouser_Tracks 89.19
Hydraulics_Flow 89.19
Track_Type 75.28
Undercarriage_Pad_Width 75.1
Stick_Length 75.27
Thumb 75.25
Pattern_Changer 75.27
Grouser_Type 75.28
Backhoe_Mounting 80.39
Blade_Type 80.1
Travel_Controls 80.1
Differential_Type 82.7
Steering_Controls 82.71
No. of columns with more than 50% Null values;  35


## Test Set Null counts 

In [12]:
NullCheck_T = rdt_T_prs.select([bround(((sum(when(isnull(c), 1).otherwise(0)))/rdt_T_prs.count()*100),2).alias(c)\
                   for c in rdt_T_prs.columns])

In [13]:
counter = 0
thresh = 50
for c in NullCheck_T.columns:
    if NullCheck_T.select(c).collect()[0][0]>thresh:
        print(c, NullCheck_T.select(c).collect()[0][0])  
        counter +=1
print('No. of columns with more than {}% Null values; '.format(thresh), counter) 

MachineHoursCurrentMeter 59.05
UsageBand 65.17
fiModelSeries 84.8
fiModelDescriptor 74.97
ProductSize 50.38
Drive_System 76.45
Forks 51.28
Pad_Type 83.05
Ride_Control 64.38
Stick 83.05
Transmission 58.72
Turbocharged 83.05
Blade_Extension 93.4
Blade_Width 93.4
Enclosure_Type 93.4
Engine_Horsepower 93.4
Pushblock 93.4
Ripper 75.74
Scarifier 93.4
Tip_Control 93.4
Tire_Size 74.77
Coupler_System 86.9
Grouser_Tracks 86.93
Hydraulics_Flow 86.93
Track_Type 73.73
Undercarriage_Pad_Width 73.7
Stick_Length 73.71
Thumb 73.7
Pattern_Changer 73.71
Grouser_Type 73.73
Backhoe_Mounting 82.37
Blade_Type 82.36
Travel_Controls 82.35
Differential_Type 81.4
Steering_Controls 81.4
No. of columns with more than 50% Null values;  35


# Print columns for ease of use 

In [88]:
print('No. of columns in the test set: ', len(rdt_prs.columns))
rdt_prs.columns

No. of columns in the test set:  55


['SalesID',
 'MachineID',
 'ModelID',
 'datasource',
 'auctioneerID',
 'YearMade',
 'MachineHoursCurrentMeter',
 'UsageBand',
 'fiModelDesc',
 'fiBaseModel',
 'fiSecondaryDesc',
 'fiModelSeries',
 'fiModelDescriptor',
 'ProductSize',
 'fiProductClassDesc',
 'state',
 'ProductGroup',
 'ProductGroupDesc',
 'Drive_System',
 'Enclosure',
 'Forks',
 'Pad_Type',
 'Ride_Control',
 'Stick',
 'Transmission',
 'Turbocharged',
 'Blade_Extension',
 'Blade_Width',
 'Enclosure_Type',
 'Engine_Horsepower',
 'Hydraulics',
 'Pushblock',
 'Ripper',
 'Scarifier',
 'Tip_Control',
 'Tire_Size',
 'Coupler',
 'Coupler_System',
 'Grouser_Tracks',
 'Hydraulics_Flow',
 'Track_Type',
 'Undercarriage_Pad_Width',
 'Stick_Length',
 'Thumb',
 'Pattern_Changer',
 'Grouser_Type',
 'Backhoe_Mounting',
 'Blade_Type',
 'Travel_Controls',
 'Differential_Type',
 'Steering_Controls',
 'logSalePrice',
 'yDay',
 'mDay',
 'wDay']

# Check out the uniqueness of values in each column
### The idea is to see if each string column is worthwhile for ML training and prediction
### Large ratios (close to 1) show that there are very few categorical variables that have been repeated
### Small ratios (close to 0) show that there are many examples to learn from for each distinct value 

###   !  Becareful about the append function  !


In [185]:
%%time 
# 1min 1s 
rep_ratio_cut_off = 0.2
maxBinCnt = 100

# The initial columns are the ones with large percentage of Nulls
clms_drop_list = []
Dis_counts     = np.array([])
Dis_ratios     = np.array([])


current_no_dropouts = len(clms_drop_list)
for c in rdt_prs.columns:
    if c != IDX and c != LBL:          # Index columns will not be used as a feature
        '''
        Calculates the ratio of the number of distinct column values to the total number of distinct columns 
        '''
        Distinct_cnt   = rdt_prs.select(c).distinct().count()
        Distinct_ratio = Distinct_cnt/rdt_prs.count()
        if Distinct_ratio>rep_ratio_cut_off or Distinct_cnt>maxBinCnt:
            print(c, '|' ,  Distinct_cnt, '|', Distinct_ratio)
            clms_drop_list.append(c)     
            np.append(Dis_counts, Distinct_cnt)
            np.append(Dis_ratios, Distinct_ratio)

print(clms_drop_list)        
print(len(clms_drop_list))      


MachineID | 341027 | 0.8501763789342475
ModelID | 5218 | 0.013008413836086008
MachineHoursCurrentMeter | 15153 | 0.03777625428482393
fiModelDesc | 4999 | 0.012462449361171704
fiBaseModel | 1950 | 0.004861327516360237
fiSecondaryDesc | 176 | 0.0004387659707073855
fiModelSeries | 123 | 0.0003066375818011842
fiModelDescriptor | 140 | 0.0003490183857899657
yDay | 360 | 0.0008974758491741975
['MachineID', 'ModelID', 'MachineHoursCurrentMeter', 'fiModelDesc', 'fiBaseModel', 'fiSecondaryDesc', 'fiModelSeries', 'fiModelDescriptor', 'yDay']
9
Wall time: 2min 10s


In [158]:
rdt_prs.columns

['SalesID',
 'MachineID',
 'ModelID',
 'datasource',
 'auctioneerID',
 'YearMade',
 'MachineHoursCurrentMeter',
 'UsageBand',
 'fiModelDesc',
 'fiBaseModel',
 'fiSecondaryDesc',
 'fiModelSeries',
 'fiModelDescriptor',
 'ProductSize',
 'fiProductClassDesc',
 'state',
 'ProductGroup',
 'ProductGroupDesc',
 'Drive_System',
 'Enclosure',
 'Forks',
 'Pad_Type',
 'Ride_Control',
 'Stick',
 'Transmission',
 'Turbocharged',
 'Blade_Extension',
 'Blade_Width',
 'Enclosure_Type',
 'Engine_Horsepower',
 'Hydraulics',
 'Pushblock',
 'Ripper',
 'Scarifier',
 'Tip_Control',
 'Tire_Size',
 'Coupler',
 'Coupler_System',
 'Grouser_Tracks',
 'Hydraulics_Flow',
 'Track_Type',
 'Undercarriage_Pad_Width',
 'Stick_Length',
 'Thumb',
 'Pattern_Changer',
 'Grouser_Type',
 'Backhoe_Mounting',
 'Blade_Type',
 'Travel_Controls',
 'Differential_Type',
 'Steering_Controls',
 'logSalePrice',
 'yDay',
 'mDay',
 'wDay']

# This segment filters out the features with large unique values


In [186]:
# CV Set
selec_lst   = list (set(rdt_prs.columns)  - set(clms_drop_list ))   # List of acceptable columns
rdt = rdt_prs.select(selec_lst)


# Test Set
selec_T_lst = list (set(rdt_T_prs.columns)  - set(clms_drop_list ))
rdt_T = rdt_T_prs.select(selec_T_lst)


In [187]:
rdt_prs.select(clms_drop_list).columns

['MachineID',
 'ModelID',
 'MachineHoursCurrentMeter',
 'fiModelDesc',
 'fiBaseModel',
 'fiSecondaryDesc',
 'fiModelSeries',
 'fiModelDescriptor',
 'yDay']

# Generate the list of string columns

In [188]:
STR_clms = [List_element[0] for List_element in rdt.dtypes if List_element[1].startswith('string')]
print('No. of string columns in the test set: ',len(STR_clms))
#STR_clms_prs

No. of string columns in the test set:  39


# Generate the list of numerical columns

In [189]:
# CV Set 
NUM_clms = [item[0] for item in rdt.dtypes if item[1].startswith('int') or\
                                              item[1].startswith('double') or\
                                              item[1].startswith('float')]
print('No. of numerical columns in the test set: ',len(NUM_clms)) 
# =============================================================================

# Test Set
NUM_T_clms = [item[0] for item in rdt_T.dtypes if item[1].startswith('int') or\
                                                item[1].startswith('double') or\
                                                item[1].startswith('float')]
print(NUM_T_clms)

No. of numerical columns in the test set:  7
['datasource', 'YearMade', 'mDay', 'SalesID', 'wDay', 'auctioneerID']


# Save to feather format 

In [65]:
# CV Set 
pd_rdt = rdt.toPandas()

# Test Set 
pd_rdt_T = rdt_T.toPandas()

In [67]:
# CV Set
pd_rdt.to_feather('pd_rdt')

# Test Set
pd_rdt_T.to_feather('pd_rdt_T')


In [9]:
rdt = spark.createDataFrame(pd.read_feather('pd_rdt'))
rdt_T = spark.createDataFrame(pd.read_feather('pd_rdt_T'))

ArrowIOError: Failed to open local file: pd_rdt , error: No such file or directory

In [8]:
rdt.printSchema()

NameError: name 'rdt' is not defined

# Convert string columns to numerical indexes

In [190]:
%%time 

# Define the indexer for all string columns
indexers        = [StringIndexer(inputCol= c, outputCol= c+'_IDX', handleInvalid='keep').fit(rdt) for c in STR_clms]


# Create the pipeline
pipe_StrToIdx   = Pipeline(stages=indexers)


# Execute the pipeline to index the string columns
# Also drop the original string columns

# -------------------------------------------------------------------------------
# CV set from train data 
sfdt_IDXed         = pipe_StrToIdx.fit(rdt).transform(rdt).drop(*STR_clms)          #sf: string-filled
# ========================================================================

# Test set 
sfdt_T_IDXed       = pipe_StrToIdx.fit(rdt_T).transform(rdt_T).drop(*STR_clms)      #sf: string-filled
# -------------------------------------------------------------------------------


#sfdt_IDXed.show()
#sfdt_T_IDXed.show()

Wall time: 15.9 s


# Handling Null Values for numerical columns
### Null counts and fractions 

##### CV and Test set total count and fraction

In [191]:
%%time 

print('CV Set---------------------')
NumRows = sfdt_IDXed.count()
print('No. of rows:', NumRows)
print("No. of Nulls", NumRows - sfdt_IDXed .dropna().count() )
print("Fraction of NULLs", np.around(1- sfdt_IDXed.dropna().count()/sfdt_IDXed .count(), 2) )
#==============================================================================================
print('\n')
print('Test Set-------------------')
NumRows = sfdt_T_IDXed.count()
print('No. of rows:', NumRows)
print("No. of Nulls", NumRows - sfdt_T_IDXed .dropna().count() )
print("Fraction of NULLs", np.around(1- sfdt_T_IDXed.dropna().count()/sfdt_T_IDXed .count(), 2) )

CV Set---------------------
No. of rows: 401125
No. of Nulls 20136
Fraction of NULLs 0.05


Test Set-------------------
No. of rows: 11573
No. of Nulls 0
Fraction of NULLs 0.0
Wall time: 1min 18s


### Null Percentages per column

#### CV Set

In [94]:
%%time
sfdt_IDXed.select([bround(((sum(when(isnull(c), 1).otherwise(0)))/NumRows*100),2).alias(c)\
                   for c in sfdt_IDXed.columns]).show()

+----------+--------+----+----+------------+------------------------+-------+-------+----+------------+----------------+-------------------+--------------------+-----------------+--------------+----------------------+----------+------------------+-------------+----------------+----------------+--------------+---------------+---------------------+----------------+-------------+-------------------+------------------+---------------+--------------------+-------------------+----------------+------------+-------------------+---------------+--------------+----------------+-------------+---------+-------------------+---------------------+-------------+---------------------------+---------------------+---------+---------------+---------+---------------------+------------------+---------+-----------+---------------+-------------+----------------+
|datasource|YearMade|mDay|yDay|logSalePrice|MachineHoursCurrentMeter|SalesID|ModelID|wDay|auctioneerID|ProductGroup_IDX|Blade_Extension_IDX|Backhoe_Mo

#### Test Set

In [95]:
%%time 
sfdt_T_IDXed.select([bround(((sum(when(isnull(c), 1).otherwise(0)))/NumRows*100),2).alias(c)\
                     for c in sfdt_T_IDXed.columns]).show()

+----------+--------+----+----+------------------------+-------+-------+----+------------+----------------+-------------------+--------------------+-----------------+--------------+----------------------+----------+------------------+-------------+----------------+----------------+--------------+---------------+---------------------+----------------+-------------+-------------------+------------------+---------------+--------------------+-------------------+----------------+------------+-------------------+---------------+--------------+----------------+-------------+---------+-------------------+---------------------+-------------+---------------------------+---------------------+---------+---------------+---------+---------------------+------------------+---------+-----------+---------------+-------------+----------------+
|datasource|YearMade|mDay|yDay|MachineHoursCurrentMeter|SalesID|ModelID|wDay|auctioneerID|ProductGroup_IDX|Blade_Extension_IDX|Backhoe_Mounting_IDX|fiModelSeries_I

### Compute Null replacements if any 
#### Figure out the catogories and their frequencies

### Replacement values  (only performed based on the CV set)

In [192]:
%%time 
# Calculate the median for numerical columns 
Med_dict = {}
for c in NUM_T_clms:
    Med_dict[c] = sfdt_IDXed.approxQuantile(c, [0.5], 0.05)[0]

Med_dict

Wall time: 52.9 s


### Replace the Null values

In [193]:
%%time 

# CV Set 
fdt = sfdt_IDXed.na.fill(Med_dict)         # fdt: filled data (includes both string and numerical values)
fdt.show()

# ===================================================================

fdt_T = sfdt_T_IDXed.na.fill(Med_dict)     # fdt: filled data (includes both string and numerical values)
fdt_T.show()


+----------+--------+----+------------------+-------+----+------------+----------------+----------------+-------------------+--------------------+--------------+----------------------+----------+------------------+-------------+----------------+----------------+--------------+---------------------+----------------+-------------+-------------------+------------------+--------------------+-------------------+----------------+------------+-------------------+---------------+--------------+----------------+-------------+---------+-------------+---------------------------+---------------------+---------+---------------+---------+---------------------+------------------+---------+-----------+---------------+-------------+
|datasource|YearMade|mDay|      logSalePrice|SalesID|wDay|auctioneerID|ProductGroup_IDX|Ride_Control_IDX|Blade_Extension_IDX|Backhoe_Mounting_IDX|Track_Type_IDX|fiProductClassDesc_IDX|Ripper_IDX|Coupler_System_IDX|UsageBand_IDX|Transmission_IDX|Grouser_Type_IDX|Blade_Type_I

+----------+--------+----+-------+----+------------+----------------+----------------+-------------------+--------------------+--------------+----------------------+----------+------------------+-------------+----------------+----------------+--------------+---------------------+----------------+-------------+-------------------+------------------+--------------------+-------------------+----------------+------------+-------------------+---------------+--------------+----------------+-------------+---------+-------------+---------------------------+---------------------+---------+---------------+---------+---------------------+------------------+---------+-----------+---------------+-------------+
|datasource|YearMade|mDay|SalesID|wDay|auctioneerID|ProductGroup_IDX|Ride_Control_IDX|Blade_Extension_IDX|Backhoe_Mounting_IDX|Track_Type_IDX|fiProductClassDesc_IDX|Ripper_IDX|Coupler_System_IDX|UsageBand_IDX|Transmission_IDX|Grouser_Type_IDX|Blade_Type_IDX|Differential_Type_IDX|Turbocharged_

#### Check for any left over Null values

In [172]:
%%time 
# CV SET
fdt.dropna().count()/fdt.count()

Wall time: 54.8 s


1.0

In [173]:
%%time 
# Test Set
fdt_T.dropna().count()/fdt_T.count()

Wall time: 6.8 s


1.0

# Rename the processed dataframes as well as the labels column

In [194]:
# CV Set 
Xy = fdt.withColumnRenamed(LBL,'Labels')

# =============================================

# Test Set (No Labels)
X = fdt_T


# This is the time to save to feather  =================

# EDA
### EDA-1 summary statistics

In [88]:
# CV Set
for c in Xy.columns:
    #XY.select(c).describe().select(bround(c,2)).show()
    Xy.select(c).describe().select('summary',bround(c,2).alias(c)).show()



+-------+--------+
|summary|FullBath|
+-------+--------+
|  count|  1460.0|
|   mean|    1.57|
| stddev|    0.55|
|    min|     0.0|
|    max|     3.0|
+-------+--------+

+-------+---------+
|summary|GrLivArea|
+-------+---------+
|  count|   1460.0|
|   mean|  1515.46|
| stddev|   525.48|
|    min|    334.0|
|    max|   5642.0|
+-------+---------+

+-------+------------+
|summary|BsmtHalfBath|
+-------+------------+
|  count|      1460.0|
|   mean|        0.06|
| stddev|        0.24|
|    min|         0.0|
|    max|         2.0|
+-------+------------+

+-------+-----------+
|summary|TotalBsmtSF|
+-------+-----------+
|  count|     1460.0|
|   mean|    1057.43|
| stddev|     438.71|
|    min|        0.0|
|    max|     6110.0|
+-------+-----------+

+-------+------------+
|summary|KitchenAbvGr|
+-------+------------+
|  count|      1460.0|
|   mean|        1.05|
| stddev|        0.22|
|    min|         0.0|
|    max|         3.0|
+-------+------------+

+-------+----------+
|summary|Ga

+-------+--------------+
|summary|MasVnrType_IDX|
+-------+--------------+
|  count|        1460.0|
|   mean|          0.53|
| stddev|          0.74|
|    min|           0.0|
|    max|           4.0|
+-------+--------------+

+-------+---------------+
|summary|Exterior1st_IDX|
+-------+---------------+
|  count|         1460.0|
|   mean|           2.03|
| stddev|           2.26|
|    min|            0.0|
|    max|           14.0|
+-------+---------------+

+-------+--------------+
|summary|CentralAir_IDX|
+-------+--------------+
|  count|        1460.0|
|   mean|          0.07|
| stddev|          0.25|
|    min|           0.0|
|    max|           1.0|
+-------+--------------+

+-------+--------------+
|summary|GarageCond_IDX|
+-------+--------------+
|  count|        1460.0|
|   mean|          0.15|
| stddev|          0.54|
|    min|           0.0|
|    max|           5.0|
+-------+--------------+

+-------+---------------+
|summary|Exterior2nd_IDX|
+-------+---------------+
|  count|

### EDA-2 vectorize

In [89]:
# Drop the index column for the CV set 
Xy_clms = Xy.columns # XY labels 
Xy_clms.remove(IDX)
# Drop the index column for the test set 
X_clms  = X.columns
X_clms.remove(IDX)


EDA_Vectorizer = VectorAssembler(inputCols= Xy_clms , outputCol='features_labels')
Vectorized_lng = EDA_Vectorizer.transform(Xy)
#Vectorized_lng.show(truncate=False)
Vectorized_sht = Vectorized_lng.select('features_labels')
Vectorized_sht.show()

+--------------------+
|     features_labels|
+--------------------+
|(80,[0,1,3,4,5,6,...|
|(80,[0,1,2,3,4,5,...|
|(80,[0,1,3,4,5,6,...|
|(80,[0,1,3,4,5,6,...|
|(80,[0,1,3,4,5,6,...|
|(80,[0,1,3,4,5,6,...|
|(80,[0,1,3,4,5,6,...|
|(80,[0,1,3,4,5,6,...|
|(80,[0,1,3,4,5,6,...|
|(80,[0,1,3,4,5,6,...|
|(80,[0,1,3,4,5,6,...|
|(80,[0,1,3,4,5,6,...|
|(80,[0,1,3,4,5,6,...|
|(80,[0,1,3,4,5,6,...|
|(80,[0,1,3,4,5,6,...|
|(80,[0,1,3,4,5,6,...|
|(80,[0,1,3,4,5,6,...|
|(80,[0,1,4,5,6,8,...|
|(80,[0,1,3,4,5,6,...|
|(80,[0,1,3,4,5,6,...|
+--------------------+
only showing top 20 rows



### EDA-3 correlations

In [90]:
# Calculate the pairwise correlations
CorrMat_Spark = Correlation.corr(Vectorized_sht,'features_labels').head()
#print("Pearson correlation matrix:\n" + str(CorrMat_Spark[0]))

#=======================================================================

# Find out the correlation matrix size

CorrMat_Spark[0]

DenseMatrix(80, 80, [1.0, 0.63, -0.0545, 0.3237, 0.1331, 0.4057, 0.5948, 0.1877, ..., 0.0082, 0.0039, 0.0255, -0.0181, -0.0148, 0.1351, -0.0221, 1.0], False)

#### Record the size

In [91]:
N_corr = 80

#### EDA-4 Convert to Pandas Dataframe For Visual Checks 

In [92]:
CorrMat_np = -2*np.ones( (N_corr,N_corr) )
for ii in range(N_corr):
    for jj in range(N_corr):
            CorrMat_np[ii,jj] = CorrMat_Spark[0][ii,jj]
CorrMat_np = np.around(CorrMat_np,2)
#CorrMat_np

# ----------------------------------------------------

CorrMat_pd = pd.DataFrame(CorrMat_np, columns = enumerate(Xy_clms))
#CorrMat_pd


,"(0, FullBath)","(1, GrLivArea)","(2, BsmtHalfBath)","(3, TotalBsmtSF)","(4, KitchenAbvGr)","(5, GarageArea)","(6, Labels)","(7, WoodDeckSF)","(8, BedroomAbvGr)","(9, PoolArea)","(10, LotArea)","(11, OverallQual)","(12, MasVnrArea)","(13, YearRemodAdd)","(14, OverallCond)","(15, LotFrontage)","(16, HalfBath)","(17, MSSubClass)","(18, BsmtFinSF1)","(19, GarageCars)","(20, TotRmsAbvGrd)","(21, GarageYrBlt)","(22, BsmtUnfSF)","(23, YearBuilt)","(24, YrSold)","(25, 3SsnPorch)","(26, MiscVal)","(27, MoSold)","(28, OpenPorchSF)","(29, 2ndFlrSF)","(30, LowQualFinSF)","(31, EnclosedPorch)","(32, ScreenPorch)","(33, BsmtFinSF2)","(34, 1stFlrSF)","(35, Fireplaces)","(36, BsmtFullBath)","(37, Heating_IDX)","(38, LandSlope_IDX)","(39, FireplaceQu_IDX)","(40, SaleCondition_IDX)","(41, RoofStyle_IDX)","(42, HouseStyle_IDX)","(43, MasVnrType_IDX)","(44, Exterior1st_IDX)","(45, CentralAir_IDX)","(46, GarageCond_IDX)","(47, Exterior2nd_IDX)","(48, BsmtFinType1_IDX)","(49, HeatingQC_IDX)","(50, Alley_IDX)","(51, RoofMatl_IDX)","(52, Functional_IDX)","(53, ExterQual_IDX)","(54, Condition1_IDX)","(55, MSZoning_IDX)","(56, LotShape_IDX)","(57, LandContour_IDX)","(58, Fence_IDX)","(59, Street_IDX)","(60, BsmtQual_IDX)","(61, Neighborhood_IDX)","(62, Foundation_IDX)","(63, GarageType_IDX)","(64, ExterCond_IDX)","(65, Utilities_IDX)","(66, BsmtCond_IDX)","(67, SaleType_IDX)","(68, BsmtExposure_IDX)","(69, BldgType_IDX)","(70, PoolQC_IDX)","(71, MiscFeature_IDX)","(72, KitchenQual_IDX)","(73, LotConfig_IDX)","(74, PavedDrive_IDX)","(75, Electrical_IDX)","(76, BsmtFinType2_IDX)","(77, GarageQual_IDX)","(78, GarageFinish_IDX)","(79, Condition2_IDX)"
0,1.00,0.63,-0.05,0.32,0.13,0.41,0.59,0.19,0.36,0.05,0.13,0.55,0.27,0.44,-0.19,0.12,0.14,0.13,0.06,0.47,0.55,0.14,0.29,0.47,-0.02,0.04,-0.01,0.06,0.26,0.42,-0.00,-0.12,-0.01,-0.08,0.38,0.24,-0.06,-0.03,-0.05,0.20,0.03,-0.01,-0.03,0.27,-0.17,-0.11,-0.14,-0.14,-0.24,-0.28,-0.01,0.01,-0.01,0.40,0.02,-0.06,0.18,-0.03,-0.19,-0.05,0.32,0.09,-0.35,-0.12,-0.09,-0.03,-0.02,0.05,0.09,0.06,0.06,-0.02,0.30,0.06,-0.14,-0.16,-0.07,-0.11,0.28,0.06
1,0.63,1.00,-0.02,0.45,0.10,0.47,0.70,0.25,0.52,0.17,0.26,0.59,0.39,0.29,-0.08,0.22,0.42,0.07,0.21,0.47,0.83,0.16,0.24,0.20,-0.04,0.02,-0.00,0.05,0.33,0.69,0.13,0.01,0.10,-0.01,0.57,0.46,0.03,-0.02,0.04,0.32,0.03,0.15,0.14,0.23,-0.01,-0.09,-0.11,-0.01,-0.15,-0.22,-0.00,0.19,0.06,0.37,0.08,-0.09,0.21,0.05,-0.10,-0.04,0.19,0.12,-0.20,-0.07,-0.05,-0.01,-0.05,0.04,0.07,-0.07,0.15,-0.02,0.31,0.06,-0.07,-0.12,-0.06,-0.06,0.19,0.09
2,-0.05,-0.02,1.00,-0.00,-0.04,-0.02,-0.01,0.04,0.05,0.02,0.05,-0.04,0.03,-0.01,0.12,-0.03,-0.01,-0.00,0.07,-0.02,-0.02,0.02,-0.10,-0.04,-0.05,0.04,-0.01,0.03,-0.03,-0.02,-0.01,-0.01,0.03,0.07,0.00,0.03,-0.15,-0.01,0.07,0.07,0.05,0.04,0.05,0.02,0.02,-0.04,-0.04,0.05,0.07,0.07,-0.04,0.02,-0.01,-0.07,-0.02,-0.06,0.06,0.03,0.03,-0.02,-0.05,0.01,0.03,-0.01,0.05,0.10,0.00,-0.00,0.02,-0.02,0.03,0.02,-0.03,0.04,-0.01,-0.02,0.06,-0.05,-0.05,-0.02
3,0.32,0.45,-0.00,1.00,-0.07,0.49,0.61,0.23,0.05,0.13,0.26,0.54,0.36,0.29,-0.17,0.24,-0.05,-0.24,0.52,0.43,0.29,0.18,0.42,0.39,-0.01,0.04,-0.02,0.01,0.25,-0.17,-0.03,-0.10,0.08,0.10,0.82,0.34,0.31,-0.14,0.03,0.20,0.04,0.16,-0.27,0.36,-0.09,-0.21,-0.15,-0.07,-0.20,-0.27,-0.11,0.20,-0.03,0.39,-0.00,-0.20,0.20,0.06,-0.11,-0.01,0.04,0.04,-0.38,-0.28,-0.13,-0.01,-0.33,0.09,-0.01,-0.13,0.10,-0.06,0.28,0.04,-0.19,-0.18,-0.11,-0.13,0.20,0.05
4,0.13,0.10,-0.04,-0.07,1.00,-0.06,-0.15,-0.09,0.20,-0.01,-0.02,-0.18,-0.04,-0.15,-0.09,0.03,-0.07,0.28,-0.08,-0.05,0.26,-0.16,0.03,-0.17,0.03,-0.02,0.06,0.03,-0.07,0.06,0.01,0.04,-0.05,-0.04,0.07,-0.12,-0.04,0.08,-0.04,-0.11,0.12,0.01,0.07,-0.06,0.10,0.25,0.14,0.12,0.10,0.14,0.02,0.02,0.03,-0.09,0.04,0.09,-0.09,0.02,-0.06,-0.01,0.07,-0.06,0.20,0.22,0.01,-0.01,0.22,0.03,0.10,0.50,-0.01,0.05,-0.10,-0.03,0.11,0.16,0.07,0.14,-0.04,0.15
5,0.41,0.47,-0.02,0.49,-0.06,1.00,0.65,0.22,0.07,0.06,0.18,0.56,0.37,0.37,-0.15,0.20,0.16,-0.10,0.30,0.88,0.34,0.56,0.18,0.48,-0.03,0.04,-0.03,0.03,0.24,0.14,-0.07,

# End of EDA ================================================

# ML Vectorize

In [195]:
# Drop the index column for the CV set 
Xy_clms = Xy.columns # XY labels 
Xy_clms.remove(IDX)
# Drop the index column for the test set 
X_clms  = X.columns
X_clms.remove(IDX)

In [196]:
# CV Set 
ML_Vectorizer   = VectorAssembler(inputCols= X_clms , outputCol='features')
ML_Vectorized   = ML_Vectorizer.transform(Xy)

# ===================================================================================

# Test Set 
ML_Vectorizer_T = VectorAssembler(inputCols= X_clms , outputCol='features')
ML_Vectorized_T = ML_Vectorizer.transform(X)


In [197]:
'''
scaler = StandardScaler(inputCol="uns_features", outputCol="features",
                        withStd=True, withMean=False)
Vec_scaled = scaler.fit(Vectorized).transform(Vectorized)
'''

'\nscaler = StandardScaler(inputCol="uns_features", outputCol="features",\n                        withStd=True, withMean=False)\nVec_scaled = scaler.fit(Vectorized).transform(Vectorized)\n'

# Define the ML Data Set

#### CV

In [198]:
%%time
cv_set = ML_Vectorized.select('features','Labels')
cv_set.show()

+--------------------+------------------+
|            features|            Labels|
+--------------------+------------------+
|[121.0,2004.0,16....|11.097410021008562|
|[121.0,1996.0,26....|10.950806546816688|
|[121.0,2001.0,26....| 9.210340371976184|
|[121.0,2001.0,19....|10.558413520275876|
|[121.0,2007.0,23....| 9.305650551780507|
|[121.0,2004.0,18....|10.184900011974314|
|[121.0,1993.0,26....|  9.95227771670556|
|[121.0,2001.0,17....|10.203592144986466|
|[121.0,2008.0,27....| 9.975808214115753|
|[121.0,1000.0,9.0...|11.082142548877775|
|[121.0,2004.0,21....|10.085809109330082|
|[121.0,1998.0,24....|10.021270588192511|
|[121.0,1998.0,20....|10.491274217438248|
|[121.0,2001.0,26....|10.325481962595504|
|[121.0,2004.0,3.0...|10.239959789157341|
|[121.0,1999.0,16....| 9.852194258148577|
|[121.0,1999.0,14....|  9.51044496442652|
|[121.0,2003.0,28....| 9.159047077588632|
|[121.0,2001.0,9.0...| 9.433483923290392|
|[121.0,1991.0,17....| 9.350102314351341|
+--------------------+------------

#### Test 

In [199]:
%%time
test_set = ML_Vectorized_T.select(IDX,'features')


Wall time: 7.02 ms


# Define the ML model 

In [200]:
mdl_RF  = RandomForestRegressor(featuresCol='features', labelCol='Labels')
#mdl_GBT = GBTClassifier(featuresCol='features', labelCol='Labels') 
# mdl_LG =  LogisticRegression(featuresCol='features', labelCol='Labels')

# Create the evaluation object
#### Options: rmse - root mean squared error (default)   |   mse - mean squared error  |  r2 - r^2 metric   |  mae - mean absolute error

In [201]:
eval1_obj = RegressionEvaluator(predictionCol='prediction', labelCol='Labels', metricName='rmse')

# Parameter Grids

In [202]:
paramGrid_RF = ParamGridBuilder()\
    .addGrid(mdl_RF.maxDepth, [5])\
    .addGrid(mdl_RF.maxBins, [100])\
    .addGrid(mdl_RF.impurity, ['variance'])\
    .addGrid(mdl_RF.minInstancesPerNode, [1])\
    .addGrid(mdl_RF.checkpointInterval, [10])\
    .addGrid(mdl_RF.subsamplingRate, [1.0])\
    .addGrid(mdl_RF.numTrees, [20])\
    .addGrid(mdl_RF.featureSubsetStrategy, ['auto'])\
    .build()

#  maxBin default is 32


# Best 


Np_RF = len(paramGrid_RF)
#Options for subsamplingRate auto, all, onethird, sqrt, log2 

# -------------------------------------
'''
paramGrid_GBT = ParamGridBuilder() \
    .addGrid(mdl_GBT.maxDepth, [5]) \
    .addGrid(mdl_GBT.maxIter,  [6,8,12,20]) \
    .addGrid(mdl_GBT.stepSize, [0.01,0.05,0.1,0.2]) \
    .build()
Np_GBT = len(paramGrid_GBT)
'''
# -------------------------------------
'''
paramGrid_LG = ParamGridBuilder() \
    .addGrid(mdl_LG.regParam, [0.01, 0.1, 0.5]) \
    .addGrid(mdl_LG.maxIter, [100,200]) \
    .build()
Np_LG = len(paramGrid_LG)'''

'\nparamGrid_LG = ParamGridBuilder()     .addGrid(mdl_LG.regParam, [0.01, 0.1, 0.5])     .addGrid(mdl_LG.maxIter, [100,200])     .build()\nNp_LG = len(paramGrid_LG)'

# Cross Validator 

In [203]:
xval_RF = CrossValidator(estimator         = mdl_RF,
                         estimatorParamMaps= paramGrid_RF,
                         evaluator         = eval1_obj,
                         numFolds          = 7,
                         parallelism       = 8)
# ---------------------------------------------------------
'''
xval_GBT = CrossValidator(estimator         = mdl_GBT,
                         estimatorParamMaps= paramGrid_GBT,
                         evaluator         = eval1_obj,
                         numFolds          = 10,
                         parallelism       = 8)
'''
'''
# ---------------------------------------------------------
xval_LG = CrossValidator(estimator         = mdl_LG,
                         estimatorParamMaps= paramGrid_LG,
                         evaluator         = eval1_obj,
                         numFolds          = 10,
                         parallelism       = 8)
'''

'\n# ---------------------------------------------------------\nxval_LG = CrossValidator(estimator         = mdl_LG,\n                         estimatorParamMaps= paramGrid_LG,\n                         evaluator         = eval1_obj,\n                         numFolds          = 10,\n                         parallelism       = 8)\n'

# Fit the model (with CV)

In [204]:
%%time
mdl_xv_RF_ftd = xval_RF.fit(cv_set)

Wall time: 14min 34s


In [205]:
#mdl_xv_GBT_ftd = xval_GBT.fit(cv_set)

In [206]:
#mdl_xv_LG_ftd = xval_LG.fit(cv_set)

# Metric AVGs for Cross Validation Across Parameter Grid 

### RF

In [207]:
print('RF CV Scores')
for pp in range(Np_RF):
    L = []
    for key in paramGrid_RF[pp]:
        L.append(paramGrid_RF[pp][key])
        
    print(round(mdl_xv_RF_ftd.avgMetrics[pp],3), L)

RF CV Scores


TypeError: Invalid argument, not a string or column: 0.4029596977381977 of type <class 'float'>. For column literals, use 'lit', 'array', 'struct' or 'create_map' function.

### GBT

In [423]:
'''print('GBT CV Scores')
for pp in range(Np_GBT):
    L = []
    for key in paramGrid_GBT[pp]:
        L.append(paramGrid_GBT[pp][key])
    print(round(mdl_xv_GBT.avgMetrics[pp],3), L)'''

"print('GBT CV Scores')\nfor pp in range(Np_GBT):\n    L = []\n    for key in paramGrid_GBT[pp]:\n        L.append(paramGrid_GBT[pp][key])\n    print(round(mdl_xv_GBT.avgMetrics[pp],3), L)"

### LG

In [424]:
'''print('LG CV Scores')
for pp in range(Np_LG):
    L = []
    for key in paramGrid_LG[pp]:
        L.append(paramGrid_LG[pp][key])
    print(round(mdl_xv_LG.avgMetrics[pp],3), L)'''

"print('LG CV Scores')\nfor pp in range(Np_LG):\n    L = []\n    for key in paramGrid_LG[pp]:\n        L.append(paramGrid_LG[pp][key])\n    print(round(mdl_xv_LG.avgMetrics[pp],3), L)"

# Test Data

In [208]:
mdl_xv_RF_preds_T = mdl_xv_RF_ftd.transform(test_set) 

In [209]:
mdl_xv_RF_preds_T = mdl_xv_RF_preds_T.withColumnRenamed('prediction','logSalePrice')
mdl_xv_RF_preds_T.columns

['SalesID', 'features', 'logSalePrice']

In [211]:
fnl = mdl_xv_RF_preds_T.select('SalesId',exp('logSalePrice').alias('SalePrice'))
fnl.show()

+-------+------------------+
|SalesId|         SalePrice|
+-------+------------------+
|1222837|23418.037046026657|
|1222839| 58317.65994875538|
|1222841| 43092.22012319888|
|1222843| 23267.16035333265|
|1222845| 46692.74808499475|
|1222847|11594.325510277944|
|1222849|21653.606464013847|
|1222850|21653.606464013847|
|1222855|27612.322412462294|
|1222863| 24103.71841874043|
|1222864|21628.101301071758|
|1222865| 24103.71841874043|
|1222867| 45430.79250928254|
|1222879| 21893.79173605241|
|1222883| 14035.93756613693|
|1222905| 52912.57153333907|
|1222908| 57887.69556485511|
|1222910|22823.523300817276|
|1222911| 34512.52611984181|
|1222917|22404.851165287724|
+-------+------------------+
only showing top 20 rows



In [213]:
%%time
Sub_PId = fnl.rdd.map(lambda r: r.SalesId).collect()
Sub_Prd = fnl.rdd.map(lambda r: r.SalePrice).collect()

Wall time: 17.4 s


In [214]:
Submission = pd.DataFrame(columns=['SalesId', 'SalePrice'])

In [215]:
Submission['SalesId'] = Sub_PId 
Submission['SalePrice']    = Sub_Prd

In [216]:
Submission 

,SalesId,SalePrice
0,1222837,23418.037046
1,1222839,58317.659949
2,1222841,43092.220123
3,1222843,23267.160353
4,1222845,46692.748085
5,1222847,11594.325510
6,1222849,21653.606464
7,1222850,21653.606464
8,1222855,27612.322412
9,1222863,24103.718419


In [165]:
Submission.to_csv('Submission.csv')

# Don't forget to remove the index column